## 更新式

$$R = \sum_{t=0}^T R_t$$

$$p_{t+1} = p_t + \Delta p_t$$
$$n_{t+1} = n_t + \Delta n_t = n_t + \alpha\{f(p_{t+1}) - n_t\}$$
$$R_{t+1} = p_{t+1} \times n_{t+1}$$
$$f(p_{t+1}) = f(p_t + \Delta p_t)$$
$$p_t^* = \argmax_{p_t} p_t \times f(p_t)$$
$$\Delta p_t = p_t^* - p_t$$

### 手順 （累積収益 $R$ の計算）

0. initialize $R (=0)$
1. initialize $T, \alpha, p_0, n_0$ 
2. calculate $R_0$ from $p_0, n_0$ 
3. for $t \in {0, 1, ..., T-1}$ 
   1. find $\Delta p_t$
   2. calculate $p_{t+1}$
   3. calculate $f(p_{t+1})$
   4. calculate $n_{t+1}$
   5. calculate $R_{t+1}$ from $p_{t+1}, n_{t+1}$ 
   6. update $R$ by adding $R_{t+1}$
4. end for
5. return $R$

In [11]:
import numpy as np


class GroundTruthDynamics2:
    def __init__(self, P_min: float, P_max: float, N_min: int, N_max: int):
        self.P_min = P_min
        self.P_max = P_max
        self.N_min = N_min
        self.N_max = N_max

    def __call__(self, p: float) -> float:
        # 線形補間
        value = self.N_max - (self.N_max - self.N_min) * (p - self.P_min) / (self.P_max - self.P_min)
        return np.clip(value, self.N_min, self.N_max)

# パラメータ設定
alpha = 0.1
P_min, P_max = 0, 5000
N_min, N_max = 0, 2000

n0 = 100
p0 = 3700

# 関数定義
f = GroundTruthDynamics2(P_min, P_max, N_min, N_max)

# シミュレーションパラメータ
num_steps = 100
n_values = np.zeros(num_steps + 1)
p_values = np.zeros(num_steps + 1)
R_values = np.zeros(num_steps + 1)

# 初期値設定
n_values[0] = n0
p_values[0] = p0
R_values[0] = p0 * n0

# シミュレーション実行
for t in range(num_steps):
    p_t = p_values[t]
    n_t = n_values[t]
    p_t_plus_1 = p_t  # delta_p_t は 0 なので、p_t_plus_1 は常に p_t と同じ
    f_p_t_plus_1 = f(p_t_plus_1)
    n_t_plus_1 = n_t + alpha * (f_p_t_plus_1 - n_t)
    R_t_plus_1 = p_t_plus_1 * n_t_plus_1
    
    # 値を保存
    p_values[t + 1] = p_t_plus_1
    n_values[t + 1] = n_t_plus_1
    R_values[t + 1] = R_t_plus_1

In [12]:
n_values

array([ 100.        ,  210.        ,  309.        ,  398.1       ,
        478.29      ,  550.461     ,  615.4149    ,  673.87341   ,
        726.486069  ,  773.8374621 ,  816.45371589,  854.8083443 ,
        889.32750987,  920.39475888,  948.355283  ,  973.5197547 ,
        996.16777923, 1016.5510013 , 1034.89590117, 1051.40631106,
       1066.26567995, 1079.63911196, 1091.67520076, 1102.50768068,
       1112.25691262, 1121.03122135, 1128.92809922, 1136.0352893 ,
       1142.43176037, 1148.18858433, 1153.3697259 , 1158.03275331,
       1162.22947798, 1166.00653018, 1169.40587716, 1172.46528945,
       1175.2187605 , 1177.69688445, 1179.92719601, 1181.9344764 ,
       1183.74102876, 1185.36692589, 1186.8302333 , 1188.14720997,
       1189.33248897, 1190.39924008, 1191.35931607, 1192.22338446,
       1193.00104601, 1193.70094141, 1194.33084727, 1194.89776254,
       1195.40798629, 1195.86718766, 1196.2804689 , 1196.65242201,
       1196.98717981, 1197.28846182, 1197.55961564, 1197.80365

In [15]:
import os
from path_info import DATA_DIR
import pandas as pd

In [16]:
df = pd.read_csv(os.path.join(DATA_DIR, 'prefecture_data_with_pairs_info_v2.csv'), index_col=0)
df.head()

,Prefecture_JP,Prefecture_EN,Prefecture_Code,Total_Population,Male_Population,Female_Population,Female_subs,Male_Subs,Total_subs,male_ratio,femal_ratio,total_ratio
0,北海道,Hokkaido,JP-01,5250000,2500000,2750000,65810,111800,177610,0.044720,0.023931,0.033830
1,青森県,Aomori,JP-02,1263000,610000,653000,10915,23650,34565,0.038770,0.016715,0.027367
2,岩手県,Iwate,JP-03,1225000,590000,635000,10980,23825,34805,0.040381,0.017291,0.028412
3,宮城県,Miyagi,JP-04,2321000,1120000,1201000,30120,52450,82570,0.046830,0.025079,0.035575
4,秋田県,Akita,JP-05,974000,470000,504000,9070,14960,24030,0.031830,0.017996,0.024671


In [22]:
df['Total_subs'].mean()

np.float64(91550.9574468085)

In [24]:
float(np.floor(df['Total_subs'].mean()))

91550.0